![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [90]:
import pandas as pd

brands = pd.read_csv("brands.csv")
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

#1. What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?
#2. Do any differences exist between the word count of a product's description and its mean rating?


In [91]:
# Merge the dataframes on 'product_id'
merged_df = pd.merge(brands, finance, on='product_id')
merged_df = pd.merge(merged_df, info, on='product_id')
merged_df = pd.merge(merged_df, reviews, on='product_id')

# Display the merged dataframe
merged_df

,product_id,brand,listing_price,sale_price,discount,revenue,product_name,description,rating,reviews
0,AH2430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G27341,Adidas,75.99,37.99,0.5,1641.17,Women's adidas Originals Sleek Shoes,"A modern take on adidas sport heritage, tailor...",3.3,24.0
2,CM0081,Adidas,9.99,5.99,0.4,398.93,Women's adidas Swim Puka Slippers,These adidas Puka slippers for women's come wi...,2.6,37.0
3,B44832,Adidas,69.99,34.99,0.5,2204.37,Women's adidas Sport Inspired Questar Ride Shoes,"Inspired by modern tech runners, these women's...",4.1,35.0
4,D98205,Adidas,79.99,39.99,0.5,5182.70,Women's adidas Originals Taekwondo Shoes,This design is inspired by vintage Taekwondo s...,3.5,72.0
...,...,...,...,...,...,...,...,...,...,...
3174,AT6100-606,Nike,0.00,64.95,0.0,0.00,Nike Tiempo Legend 8 Academy TF,The Nike Tiempo Legend 8 Academy TF takes the ...,0.0,0.0
3175,CT9155-063,Nike,0.00,139.95,0.0,251.91,Nike React Metcon AMP,The Nike React Metcon AMP takes the stability ...,3.0,1.0
3176,CI1236-100,Nike,159.95,127.97,0.0,230.35,Air Jordan 8 Retro,The Air Jordan 8 Retro recaptures the memorabl...,5.0,1.0
3177,AH6799-300,Nike,0.00,169.95,0.0,1223.64,Nike Air Max 98,The Nike Air Max 98 features the OG design lin...,4.0,4.0


In [92]:
#Drop null values
df = merged_df.dropna()

In [93]:
# Label products' price into quartiles
import pandas as pd

# Ensure df is not None and 'listing_price' column exists
if df is not None and 'listing_price' in df.columns:
    df['price_label'] = pd.qcut(df['listing_price'], q=4, labels=['Elite', 'Expensive', 'Average', 'Budget']).astype(str)
else:
    print("DataFrame is None or 'listing_price' column is missing")

df

,product_id,brand,listing_price,sale_price,discount,revenue,product_name,description,rating,reviews,price_label
1,G27341,Adidas,75.99,37.99,0.5,1641.17,Women's adidas Originals Sleek Shoes,"A modern take on adidas sport heritage, tailor...",3.3,24.0,Average
2,CM0081,Adidas,9.99,5.99,0.4,398.93,Women's adidas Swim Puka Slippers,These adidas Puka slippers for women's come wi...,2.6,37.0,Elite
3,B44832,Adidas,69.99,34.99,0.5,2204.37,Women's adidas Sport Inspired Questar Ride Shoes,"Inspired by modern tech runners, these women's...",4.1,35.0,Average
4,D98205,Adidas,79.99,39.99,0.5,5182.70,Women's adidas Originals Taekwondo Shoes,This design is inspired by vintage Taekwondo s...,3.5,72.0,Average
5,B75586,Adidas,47.99,19.20,0.6,1555.20,Women's adidas Sport Inspired Duramo Lite 2.0 ...,Refine your interval training in these women's...,1.0,45.0,Expensive
...,...,...,...,...,...,...,...,...,...,...,...
3174,AT6100-606,Nike,0.00,64.95,0.0,0.00,Nike Tiempo Legend 8 Academy TF,The Nike Tiempo Legend 8 Academy TF takes the ...,0.0,0.0,Elite
3175,CT9155-063,Nike,0.00,139.95,0.0,251.91,Nike React Metcon AMP,The Nike React Metcon AMP takes the stability ...,3.0,1.0,Elite
3176,CI1236-100,Nike,159.95,127.97,0.0,230.35,Air Jordan 8 Retro,The Air Jordan 8 Retro recaptures the memorabl...,5.0,1.0,Budget
3177,AH6799-300,Nike,0.00,169.95,0.0,1223.64,Nike Air Max 98,The Nike Air Max 98 features the OG design lin...,4.0,4.0,Elite


In [94]:

# Define a custom sorting function
def custom_sort_key(price_label):
    sort_order = {'Budget': 0, 'Average': 1, 'Expensive': 2, 'Elite': 3}
    return sort_order[price_label]

adidas_vs_nike = df.groupby(by=['brand', 'price_label']).agg(
    num_products=('product_id', 'count'),
    mean_revenue=('revenue', 'mean')
).round(2)

# Sort the DataFrame using the custom sort key for 'price_label'
adidas_vs_nike = adidas_vs_nike.reset_index().sort_values(
    by=['brand', 'price_label'],
    key=lambda x: x.map(custom_sort_key) if x.name == 'price_label' else x
).set_index(['brand', 'price_label'])

adidas_vs_nike

num_products  mean_revenue
brand  price_label                            
Adidas Budget                587       8302.78
       Average               759       4621.56
       Expensive             655       3035.30
       Elite                 574       2015.68
Nike   Budget                130       1367.45
       Average                47        500.56
       Expensive               8        675.59
       Elite                 357       1596.33

In [95]:
#Finding the length of each product description
df['description_length'] = df['description'].str.len()

#Creating bins for description lengths (a list of upper word litmits for each bin)
limits = [0,100,200,300,400,500,600,700]
labels = ['100','200','300','400','500','600','700']
df['description_length'] = pd.cut(df['description_length'], bins=limits, labels = labels)
df

,product_id,brand,listing_price,sale_price,discount,revenue,product_name,description,rating,reviews,price_label,description_length
1,G27341,Adidas,75.99,37.99,0.5,1641.17,Women's adidas Originals Sleek Shoes,"A modern take on adidas sport heritage, tailor...",3.3,24.0,Average,200
2,CM0081,Adidas,9.99,5.99,0.4,398.93,Women's adidas Swim Puka Slippers,These adidas Puka slippers for women's come wi...,2.6,37.0,Elite,200
3,B44832,Adidas,69.99,34.99,0.5,2204.37,Women's adidas Sport Inspired Questar Ride Shoes,"Inspired by modern tech runners, these women's...",4.1,35.0,Average,300
4,D98205,Adidas,79.99,39.99,0.5,5182.70,Women's adidas Originals Taekwondo Shoes,This design is inspired by vintage Taekwondo s...,3.5,72.0,Average,300
5,B75586,Adidas,47.99,19.20,0.6,1555.20,Women's adidas Sport Inspired Duramo Lite 2.0 ...,Refine your interval training in these women's...,1.0,45.0,Expensive,300
...,...,...,...,...,...,...,...,...,...,...,...,...
3174,AT6100-606,Nike,0.00,64.95,0.0,0.00,Nike Tiempo Legend 8 Academy TF,The Nike Tiempo Legend 8 Academy TF takes the ...,0.0,0.0,Elite,200
3175,CT9155-063,Nike,0.00,139.95,0.0,251.91,Nike React Metcon AMP,The Nike React Metcon AMP takes the stability ...,3.0,1.0,Elite,400
3176,CI1236-100,Nike,159.95,127.97,0.0,230.35,Air Jordan 8 Retro,The Air Jordan 8 Retro recaptures the memorabl...,5.0,1.0,Budget,300
3177,AH6799-300,Nike,0.00,169.95,0.0,1223.64,Nike Air Max 98,The Nike Air Max 98 features the OG design lin...,4.0,4.0,Elite,300


In [96]:
description_lengths = df.groupby(by = 'description_length').agg(
    mean_rating = ('rating','mean'),
    num_reviews = ('reviews','count')).round(2)

description_lengths

,mean_rating,num_reviews
description_length,,
100,2.26,7
200,3.19,526
300,3.28,1785
400,3.29,651
500,3.35,118
600,3.12,15
700,3.65,15


In [97]:
print(adidas_vs_nike)


                    num_products  mean_revenue
brand  price_label                            
Adidas Budget                587       8302.78
       Average               759       4621.56
       Expensive             655       3035.30
       Elite                 574       2015.68
Nike   Budget                130       1367.45
       Average                47        500.56
       Expensive               8        675.59
       Elite                 357       1596.33


In [98]:
print(description_lengths)

                    mean_rating  num_reviews
description_length                          
100                        2.26            7
200                        3.19          526
300                        3.28         1785
400                        3.29          651
500                        3.35          118
600                        3.12           15
700                        3.65           15
